In [1]:
import dlib
import os
import numpy as np


detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor("shape_predictor_5_face_landmarks.dat")
model = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")

In [3]:
tp, total = 0,0
incorrect = []

In [20]:
# Comparison using Dlib

src_path = os.path.join("Dataset", "Face Dataset")

def findEcledianDistance(src_rep, test_rep):
    dist = src_rep - test_rep
    dist = np.sum(np.multiply(dist, dist))
    dist =np.sqrt(dist)
    return dist

for i in range(1, 103):
    baseline_path = os.path.join(src_path, str(i))
    baseline = dlib.load_rgb_image(os.path.join(baseline_path, str(i) + ".jpg"))
    
    dist = []
    
    for j in range(1, 11):
        filtered_path = os.path.join(baseline_path, "Filtered")
        filtered_img_path = os.path.join(filtered_path, str(i) + "_" + str(j) + ".jpg")
        filtered = dlib.load_rgb_image(filtered_img_path)

        baseline_detected = detector(baseline, 1)
        filtered_detected = detector(filtered, 1)

        baseline_shape = sp(baseline, baseline_detected[0])
        filtered_shape = sp(filtered, filtered_detected[0])

        baseline_alligned = dlib.get_face_chip(baseline, baseline_shape)
        filtered_alligned = dlib.get_face_chip(filtered, filtered_shape)

        baseline_rep = np.array(model.compute_face_descriptor(baseline_alligned))
        filtered_rep = np.array(model.compute_face_descriptor(filtered_alligned))
        
        if(findEcledianDistance(baseline_rep, filtered_rep) < 0.6): #
            tp += 1
        else:
            incorrect.append(filtered_img_path)
        total += 1

#         dist.append(findEcledianDistance(baseline_rep, filtered_rep) < 0.5)
# #         print(dist)
#     print(dist)

RuntimeError: Unable to open file: Face Dataset\51\Filtered\51_1.jpg

In [15]:
print("Percentage of correct predictions for Dlib is {}%".format((tp/total)*100))

Percentage of correct predictions for Dlib is 89.568345323741%


In [18]:
incorrect

'Face Dataset\\51\\Filtered51_1.jpg'

In [27]:
conda install -c conda-forge retina-face

^C

Note: you may need to restart the kernel to use updated packages.


In [29]:
!pip install deepface

  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115942 sha256=64194d120200b84297b2650c39cc2983797cb7fa2b8097c94d3f95ceec040f52
  Stored in directory: c:\users\yyash\appdata\local\pip\cache\wheels\2a\93\86\8cd17bc6c40fb605c3ac549d0b860ef7e84ee5f67bf01a3287
Successfully built fire


In [7]:
# Comparison using RetinaFace

from retinaface import RetinaFace
import matplotlib.pyplot as plt
from deepface import DeepFace

for i in range(1, 103):
    baseline_path = os.path.join(src_path, str(i))

    filtered_path = os.path.join(baseline_path, "Filtered")
    result = []
    for j in range(1, 11):
        obj = DeepFace.verify(os.path.join(baseline_path, str(i) + ".jpg"), 
                      os.path.join(filtered_path, str(i) + "_" + str(j) + ".jpg"), 
                      model_name = 'ArcFace', 
                      detector_backend = 'retinaface')
        print(obj)
        result.append(obj['verified'])
    print("Result = ")
    print(result)
    break

1/1 [==============================] - 0s 188ms/step
{'verified': True, 'distance': 0.14570072112206833, 'threshold': 0.68, 'model': 'ArcFace', 'detector_backend': 'retinaface', 'similarity_metric': 'cosine'}
1/1 [==============================] - 0s 178ms/step
{'verified': True, 'distance': 0.4109085271638956, 'threshold': 0.68, 'model': 'ArcFace', 'detector_backend': 'retinaface', 'similarity_metric': 'cosine'}
1/1 [==============================] - 0s 190ms/step
{'verified': True, 'distance': 0.2328102207383369, 'threshold': 0.68, 'model': 'ArcFace', 'detector_backend': 'retinaface', 'similarity_metric': 'cosine'}
1/1 [==============================] - 0s 190ms/step
{'verified': True, 'distance': 0.2521511389622346, 'threshold': 0.68, 'model': 'ArcFace', 'detector_backend': 'retinaface', 'similarity_metric': 'cosine'}
1/1 [==============================] - 0s 196ms/step
{'verified': True, 'distance': 0.12658491393045168, 'threshold': 0.68, 'model': 'ArcFace', 'detector_backend': 're

In [49]:
!pip install keras_vggface

In [11]:
# Comparison using MTCNN
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
# from keras.utils.layer_utils import preprocess_input
import sys

def extract_face(filename, required_size=(224, 224)):
    pixels = pyplot.imread(filename)
    detector = MTCNN()
    
    results = detector.detect_faces(pixels)
    
    x1, y1, width, height = results[0]['box']
    x2, y2 = x1 + width, y1 + height
    face = pixels[y1:y2, x1:x2]
    
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    return face_array
 
# extract faces and calculate face embeddings for a list of photo files
def get_embeddings(filenames):
    # extract faces
    faces = [extract_face(f) for f in filenames]
    
    # convert into an array of samples
    samples = asarray(faces, 'float32')
    
    # prepare the face for the model, e.g. center pixels
    samples = preprocess_input(samples, version=2)
    
    # create a vggface model
    model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
    # perform prediction
    yhat = model.predict(samples)
    return yhat
 
# determine if a candidate face is a match for a known face
def is_match(known_embedding, candidate_embedding, thresh=0.5):
    # calculate distance between embeddings
    score = cosine(known_embedding, candidate_embedding)
    if score <= thresh:
        return (score, thresh)
    return (score, thresh)
 
    
for i in range(1, 103):
    baseline_path = os.path.join(src_path, str(i))
    
    filtered_path = os.path.join(baseline_path, "Filtered")
    result_mtcnn = []
    for j in range(1, 11):
        filenames = [ os.path.join(baseline_path, str(i) + ".jpg"), os.path.join(filtered_path, str(i) + "_" + str(j) + ".jpg") ]

        embeddings = get_embeddings()

        result_mtcnn.append(is_match(embeddings[0], embeddings[1]))
#         obj = DeepFace.verify(os.path.join(baseline_path, str(i) + ".jpg"), 
#                        os.path.join(filtered_path, str(i) + "_" + str(j) + ".jpg"), 
#                        detector_backend = 'mtcnn')
#         print(obj)
#         result_mtcnn.append(obj['distance'])
    print(result_mtcnn)
    break
    


vgg_face_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/vgg_face_weights.h5
To: C:\Users\yyash\.deepface\weights\vgg_face_weights.h5
100%|███████████████████████████████████████████████████████████████████████████████| 580M/580M [09:31<00:00, 1.02MB/s]


1/1 [==============================] - 0s 408ms/step
{'verified': True, 'distance': 0.13627822640830867, 'threshold': 0.4, 'model': 'VGG-Face', 'detector_backend': 'mtcnn', 'similarity_metric': 'cosine'}
1/1 [==============================] - 0s 421ms/step
{'verified': False, 'distance': 0.4883955299639531, 'threshold': 0.4, 'model': 'VGG-Face', 'detector_backend': 'mtcnn', 'similarity_metric': 'cosine'}
1/1 [==============================] - 0s 372ms/step
{'verified': True, 'distance': 0.2692537125570227, 'threshold': 0.4, 'model': 'VGG-Face', 'detector_backend': 'mtcnn', 'similarity_metric': 'cosine'}
1/1 [==============================] - 0s 433ms/step
{'verified': True, 'distance': 0.3214640091143077, 'threshold': 0.4, 'model': 'VGG-Face', 'detector_backend': 'mtcnn', 'similarity_metric': 'cosine'}
1/1 [==============================] - 0s 482ms/step
{'verified': True, 'distance': 0.12402208948796034, 'threshold': 0.4, 'model': 'VGG-Face', 'detector_backend': 'mtcnn', 'similarity_m

{'verified': True, 'distance': 0.347435548695208, 'threshold': 0.4, 'model': 'VGG-Face', 'detector_backend': 'mtcnn', 'similarity_metric': 'cosine'}
1/1 [==============================] - 0s 374ms/step
{'verified': True, 'distance': 0.10225699126792243, 'threshold': 0.4, 'model': 'VGG-Face', 'detector_backend': 'mtcnn', 'similarity_metric': 'cosine'}
[0.13627822640830867, 0.4883955299639531, 0.2692537125570227, 0.3214640091143077, 0.12402208948796034, 0.18921249767213544, 0.34313331288327353, 0.48339265768052897, 0.347435548695208, 0.10225699126792243]
